# Lakebase Data Querying & Analysis

This notebook demonstrates how to query and analyze data from your Lakebase database.

In [2]:
import psycopg2
import pandas as pd
from databricks.sdk import WorkspaceClient
import uuid
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Configuration

In [3]:
# Configuration
INSTANCE_NAME = "rb-demo-lakebase"
PROFILE = "az-demo"
USER_NAME = "rohit.bhagwat@databricks.com"

In [4]:
# Get instance and generate credentials
w = WorkspaceClient(profile=PROFILE)
instance = w.database.get_database_instance(name=INSTANCE_NAME)
cred = w.database.generate_database_credential(
    request_id=str(uuid.uuid4()), 
    instance_names=[INSTANCE_NAME]
)

print(f"✅ Connected to instance: {instance.name}")

✅ Connected to instance: rb-demo-lakebase


In [5]:
# Connect to PostgreSQL
conn = psycopg2.connect(
    host=instance.read_write_dns,
    dbname="databricks_postgres",
    user=USER_NAME,
    password=cred.token,
    sslmode="require"
)

print("✅ Connected to PostgreSQL database")

✅ Connected to PostgreSQL database


## Basic Data Exploration

In [6]:
# Get all coffee shops
with conn.cursor() as cur:
    cur.execute("SELECT * FROM coffee_operations.coffee_shops")
    rows = cur.fetchall()
    
print(f"📊 Found {len(rows)} coffee shops")
print("\nFirst 3 shops:")
for i, row in enumerate(rows[:3]):
    print(f"  {i+1}. {row[1]} - {row[2]}, {row[3]}")

📊 Found 5 coffee shops

First 3 shops:
  1. Brew & Bean Downtown - San Francisco, CA
  2. Mocha Haven Central - New York, NY
  3. Espresso Corner - Seattle, WA


## Geographic Analysis

In [7]:
# Analyze shops by country
with conn.cursor() as cur:
    cur.execute("""
        SELECT country, COUNT(*) as shop_count, AVG(seating_capacity) as avg_capacity
        FROM coffee_operations.coffee_shops
        GROUP BY country
        ORDER BY shop_count DESC
    """)
    
    country_stats = cur.fetchall()
    
print("🌍 Shops by Country:")
for country, count, avg_cap in country_stats:
    print(f"   {country}: {count} shops, avg {avg_cap:.0f} seats")

🌍 Shops by Country:
   United States: 3 shops, avg 35 seats
   France: 1 shops, avg 50 seats
   Japan: 1 shops, avg 35 seats


## Premium vs Standard Locations

In [8]:
# Compare premium vs standard locations
with conn.cursor() as cur:
    cur.execute("""
        SELECT 
            is_premium_location,
            COUNT(*) as shop_count,
            AVG(seating_capacity) as avg_capacity,
            MIN(seating_capacity) as min_capacity,
            MAX(seating_capacity) as max_capacity
        FROM coffee_operations.coffee_shops
        GROUP BY is_premium_location
        ORDER BY is_premium_location DESC
    """)
    
    premium_stats = cur.fetchall()
    
print("⭐ Premium vs Standard Analysis:")
for is_premium, count, avg_cap, min_cap, max_cap in premium_stats:
    type_label = "Premium" if is_premium else "Standard"
    print(f"\n{type_label} Locations ({count} shops):")
    print(f"   Average capacity: {avg_cap:.1f} seats")
    print(f"   Capacity range: {min_cap} - {max_cap} seats")

⭐ Premium vs Standard Analysis:

Premium Locations (3 shops):
   Average capacity: 42.3 seats
   Capacity range: 32 - 50 seats

Standard Locations (2 shops):
   Average capacity: 31.5 seats
   Capacity range: 28 - 35 seats


## Time Zone Distribution

In [9]:
# Analyze shops by time zone
with conn.cursor() as cur:
    cur.execute("""
        SELECT 
            time_zone,
            COUNT(*) as shop_count,
            STRING_AGG(shop_name, ', ' ORDER BY shop_name) as shop_names
        FROM coffee_operations.coffee_shops
        GROUP BY time_zone
        ORDER BY shop_count DESC
    """)
    
    tz_stats = cur.fetchall()
    
print("🕐 Shops by Time Zone:")
for tz, count, names in tz_stats:
    print(f"\n{tz} ({count} shops):")
    print(f"   {names}")

🕐 Shops by Time Zone:

America/Los_Angeles (2 shops):
   Brew & Bean Downtown, Espresso Corner

America/New_York (1 shops):
   Mocha Haven Central

Asia/Tokyo (1 shops):
   Tokyo Roast

Europe/Paris (1 shops):
   Café Paris


## Load Data into Pandas for Advanced Analysis

In [10]:
# Load all data into pandas DataFrame
query = """
    SELECT shop_id, shop_name, city, state_province, country, 
           time_zone, latitude, longitude, seating_capacity, 
           is_premium_location, is_active, created_at
    FROM coffee_operations.coffee_shops
    ORDER BY shop_name
"""

df = pd.read_sql_query(query, conn)
print(f"📊 Loaded {len(df)} rows into DataFrame")
df.head()

/var/folders/70/x9xt8g1153qcq89qgj06ymcm0000gp/T/ipykernel_74654/1035384310.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


📊 Loaded 5 rows into DataFrame


,shop_id,shop_name,city,state_province,country,time_zone,latitude,longitude,seating_capacity,is_premium_location,is_active,created_at
0,CS001,Brew & Bean Downtown,San Francisco,CA,United States,America/Los_Angeles,37.7749,-122.4194,45,True,True,2025-08-15 20:29:11.132816
1,CS004,Café Paris,Paris,Île-de-France,France,Europe/Paris,48.8566,2.3522,50,True,True,2025-08-15 21:00:33.721063
2,CS003,Espresso Corner,Seattle,WA,United States,America/Los_Angeles,47.6062,-122.3321,28,False,True,2025-08-15 20:29:11.200018
3,CS002,Mocha Haven Central,New York,NY,United States,America/New_York,40.7128,-74.0060,32,True,True,2025-08-15 20:29:11.168116
4,CS005,Tokyo Roast,Tokyo,Tokyo,Japan,Asia/Tokyo,35.6762,139.6503,35,False,True,2025-08-15 21:00:33.757406


In [11]:
# Summary statistics
print("📈 Summary Statistics:")
print(df.describe())

print("\n🌍 Geographic Coverage:")
print(f"   Countries: {df['country'].nunique()}")
print(f"   Cities: {df['city'].nunique()}")
print(f"   Time Zones: {df['time_zone'].nunique()}")

📈 Summary Statistics:
        latitude   longitude  seating_capacity                     created_at
count   5.000000    5.000000          5.000000                              5
mean   42.125340  -35.351000         38.000000  2025-08-15 20:41:44.195883776
min    35.676200 -122.419400         28.000000     2025-08-15 20:29:11.132816
25%    37.774900 -122.332100         32.000000  2025-08-15 20:29:11.168115968
50%    40.712800  -74.006000         35.000000  2025-08-15 20:29:11.200017920
75%    47.606200    2.352200         45.000000  2025-08-15 21:00:33.721062912
max    48.856600  139.650300         50.000000     2025-08-15 21:00:33.757406
std     5.870747  110.326576          9.192388                            NaN

🌍 Geographic Coverage:
   Countries: 3
   Cities: 5
   Time Zones: 4


In [12]:
# Close connection
conn.close()
print("\n✅ Query session complete!")


✅ Query session complete!


## What You've Accomplished

✅ **Connected** to Lakebase PostgreSQL database  
✅ **Explored** coffee shop data across multiple countries  
✅ **Analyzed** premium vs standard location patterns  
✅ **Examined** geographic and time zone distributions  
✅ **Loaded** data into pandas for advanced analytics  

This demonstrates how Lakebase provides both operational database capabilities and analytical querying power in one unified platform!